In [1]:
#Create one hot sparse matrix for phone brand and device model

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

gender_age_test = pd.read_csv('./input/gender_age_test.csv', index_col='device_id')
gender_age_train = pd.read_csv('./input/gender_age_train.csv', index_col='device_id')
phone_brand_device_model = pd.read_csv('./input/phone_brand_device_model.csv')
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id', keep='first').set_index('device_id')

In [3]:
phone_brand_device_model.head()

,phone_brand,device_model
device_id,,
-8890648629457979026,小米,红米
1277779817574759137,小米,MI 2
5137427614288105724,三星,Galaxy S4
3669464369358936369,SUGAR,时尚手机
-5019277647504317457,三星,Galaxy Note 2


In [4]:
from sklearn.preprocessing import LabelEncoder
brand_encoder = LabelEncoder().fit(phone_brand_device_model.phone_brand)
phone_brand_device_model['brand_encode'] = brand_encoder.transform(phone_brand_device_model.phone_brand)

device_encoder = LabelEncoder().fit(phone_brand_device_model.device_model)
phone_brand_device_model['device_encode'] = device_encoder.transform(phone_brand_device_model.device_model)

In [5]:
print(len(brand_encoder.classes_))
print(len(device_encoder.classes_))

131
1599


In [6]:
gender_age_train['trainrow'] = np.arange(gender_age_train.shape[0])

In [7]:
result = phone_brand_device_model.merge(gender_age_train[['trainrow']], left_index=True, right_index=True, how='left')
result.dropna(subset=['trainrow'], inplace=True)

In [8]:
result.head()

,phone_brand,device_model,brand_encode,device_encode,trainrow
device_id,,,,,
-8890648629457979026,小米,红米,51,1517,16873.0
1277779817574759137,小米,MI 2,51,749,72877.0
-5019277647504317457,三星,Galaxy Note 2,15,536,11537.0
3238009352149731868,华为,Mate,31,774,29664.0
-5827952925479472594,小米,MI One Plus,51,758,8233.0


In [9]:
from scipy.sparse import csr_matrix, hstack
brand_train = csr_matrix((
        np.ones(result.shape[0]), 
        (result.trainrow, result.brand_encode)), 
        shape=(gender_age_train.shape[0], len(brand_encoder.classes_)))

In [10]:
device_train = csr_matrix((
        np.ones(result.shape[0]),
        (result.trainrow, result.device_encode)),
        shape=(gender_age_train.shape[0], len(device_encoder.classes_)))

In [32]:
np.savez('intermediates/brand_train', \
         data = brand_train.data, \
         indices = brand_train.indices, \
         indptr = brand_train.indptr,)

In [33]:
np.savez('intermediates/device_train', \
         data=device_train.data, \
         indices=device_train.indices, \
         indptr=device_train.indptr)